In [1]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
import numpy as np
import tscv

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
x = np.random.rand(10000000, 10)
y = np.random.rand(10000000)

In [3]:
# Define custom preprocessor
class StandardScaler:
    def fit(self, x, y):
        self.mean_ = np.mean(x, axis=0)
        self.std_ = np.std(x, axis=0)

    def transform(self, x, y):
        return ((x - self.mean_) / self.std_, y)
    
    def fit_transform(self, x, y):
        self.fit(x, y)
        return self.transform(x, y)

In [4]:
# Define custom model
class CustomLinearModel:
    def fit(self, x, y):
        self.coef_ = np.linalg.pinv(x) @ y

    def predict(self, x):
        return x @ self.coef_

In [5]:
# Define custom metrics
class MeanSquaredError:
    def evaluate(self, y_obs, y_pred):
        return np.mean((y_obs - y_pred) ** 2)

class MeanAbsoluteError:
    def evaluate(self, y_obs, y_pred):
        return np.mean(np.abs(y_obs - y_pred))

In [6]:
# Set up cross-validation
cross_validator = tscv.TimeSeriesCrossValidator(
    train_starts=[0, 20000, 40000],
    train_sizes=[100000, 100000, 100000],
    test_sizes=[20000, 20000, 20000],
    preprocessor=StandardScaler(),
    max_workers=1
)

In [7]:
# Define model and preprocessor
model = CustomLinearModel()

In [8]:
# Define metrics
metrics = [MeanSquaredError(), MeanAbsoluteError()]

In [9]:
# Evaluate model
evaluations = cross_validator.cross_validate(model, x, y, metrics)
print("Evaluation Results:")
for evaluation in evaluations:
    print(evaluation)

Evaluation Results:
{'MeanSquaredError': 0.33005341232878826, 'MeanAbsoluteError': 0.49564664352651217}
{'MeanSquaredError': 0.33235560326040625, 'MeanAbsoluteError': 0.4998735631196291}
{'MeanSquaredError': 0.3353055420441354, 'MeanAbsoluteError': 0.5014131826589037}
